# Amharic Hate Speech Detector using SVM

## Preprocessing

### Importing Dataset

In [1]:
import re
import math
import pandas
import string
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
%matplotlib inline

In [3]:
DATASET_FILEPATH = "datasets/dataset.xls"
raw_data = pandas.read_excel(DATASET_FILEPATH, sheet_name='sheet')

raw_data = shuffle(raw_data)
raw_data.head()

,Comment No.,0,Comment,Like,(view source),Timestamp
2789,NaN,0.0,ዋሂድ ቤተ አማራ ki ki ke beltsegna gar wedefit!,1,view comment,"Thursday, April 9, 2020 1:03pm GMT"
1757,18.0,0.0,"The ""liqawint"" of hundreds of years ago had a ...",0,view comment,"Tuesday, April 21, 2020 8:15pm GMT"
2412,291.0,0.0,Conjo nat congrats,0,view comment,"Monday, April 20, 2020 2:31pm GMT"
2949,21.0,0.0,በየገጠር ከተማው ከምፕ እየሰሩ ሕዝብ እየቀጠቀጡ ነው። ለምርጫም ተዘጋጅተዋል።,1,view comment,"Thursday, April 23, 2020 1:58pm GMT"
1735,NaN,0.0,abbaamaccaa ስምህ እራሱ እንዳይገባህ ተደርጎ ስለተፈጠረ ከሚመጥኑ...,1,view comment,"Tuesday, April 21, 2020 8:44pm GMT"


### Extracting Data

In [4]:
labels = raw_data.iloc[:, 1]
comments = raw_data.iloc[:, 2]

### Remove Alphabets

In [5]:
comments = [re.sub(r"[A-Za-z]", "", str(comment)) for comment in comments]

### Remove Numbers

In [6]:
comments = [re.sub('[0-9]', "", str(comment)) for comment in comments]

### Remove Punctuations

In [7]:
punctuations = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~፡።፣፥¡"
comments = [str(comment).translate(str.maketrans('', '', punctuations)) for comment in comments]

### Remove Whitespaces

In [8]:
comments = [(" ".join([ word for word in str(comment).split()])) for comment in comments]

### Remove Empty Comments

In [9]:
X = []
Y = []

number_of_comments = len(comments)
for i in range(number_of_comments):
    if comments[i].strip() and not math.isnan(float(labels[i])):
        X.append(comments[i])
        Y.append(labels[i])

## Bag of Words Encoding

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
count_vectorizer = CountVectorizer(max_features=1000, min_df=5, max_df=0.7)
X = count_vectorizer.fit_transform(X).toarray()

In [12]:
print("Training Dataset Shape: ", X.shape)
print("Training Label Shape: ", len(Y))

Training Dataset Shape:  (2490, 579)
Training Label Shape:  2490


## Split Dataset

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [14]:
print("X_train Shape: ", X_train.shape)
print("X_test Shape: ", X_test.shape)

X_train Shape:  (1992, 579)
X_test Shape:  (498, 579)


In [15]:
print("Y_train Shape: ", len(Y_train))
print("Y_test Shape: ", len(Y_test))

Y_train Shape:  1992
Y_test Shape:  498


## SVM

In [16]:
from sklearn.svm import SVC

In [17]:
classifier = SVC(kernel='linear')
classifier.fit(X_train, Y_train) 

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

## Evaluate Model

In [18]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [19]:
Y_pred = classifier.predict(X_test)

### Confusion Matrix

In [20]:
print(confusion_matrix(Y_test, Y_pred))

[[476   2]
 [ 20   0]]


### Classification Report

In [21]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       478
         1.0       0.00      0.00      0.00        20

    accuracy                           0.96       498
   macro avg       0.48      0.50      0.49       498
weighted avg       0.92      0.96      0.94       498



### Accuracy Score

In [22]:
print(accuracy_score(Y_test, Y_pred))

0.9558232931726908
